# DX 704 Week 9 Project

This week's project will build an email spam classifier based on the Enron email data set.
You will perform your own feature extraction, and use naive Bayes to estimate the probability that a particular email is spam or not.
Finally, you will review the tradeoffs from different thresholds for automatically sending emails to the junk folder.

The full project description and a template notebook are available on GitHub: [Project 9 Materials](https://github.com/bu-cds-dx704/dx704-project-09).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [1]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-01 21:24:01--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-01 21:24:01--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip’

enron_spam_data.zip 100%[===================>]  14.92M  --.-KB/s    in 0.1s    

2025-11-01 21:24:02 (118 MB/s) - ‘enron_spam

In [2]:
import pandas as pd

In [3]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [ ]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Design a Feature Extractor

Design a feature extractor for this data set and write out two files of features based on the text.
Don't forget that both the Subject and Message columns are relevant sources of text data.
For each email, you should count the number of repetitions of each feature present.
The auto-grader will assume that you are using a multinomial distribution in the following problems.

In [56]:
import json
import re
import numpy as np
from collections import Counter

In [4]:
# YOUR CHANGES HERE

def tokenize(text):
    return re.findall(r"[a-zA-Z]{2,}", text.lower())

In [5]:
def extract_features(text):
    tokens = tokenize(text)
    counts = Counter(tokens)
    return json.dumps(counts, sort_keys=True)

Assign a row to the test data set if `Message ID % 30 == 0` and assign it to the training data set otherwise.
Write two files, "train-features.tsv" and "test-features.tsv" with two columns, Message ID and features_json.
The features_json column should contain a JSON dictionary where the keys are your feature names and the values are integer feature values.
This will give us a sparse feature representation.


In [28]:
# YOUR CHANGES HERE

df = enron_spam_data.copy()
df["text"] = df["Subject"].fillna("") + " " + df["Message"].fillna("")

In [29]:
train_df = df[df["Message ID"] % 30 != 0].copy()
test_df  = df[df["Message ID"] % 30 == 0].copy()

In [30]:
train_df["features_json"] = train_df["text"].apply(extract_features)
test_df["features_json"]  = test_df["text"].apply(extract_features)

In [31]:
train_out = train_df[["Message ID", "features_json"]]
test_out  = test_df[["Message ID", "features_json"]]

In [32]:
train_out

,Message ID,features_json
1,1,"{""about"": 1, ""advises"": 1, ""am"": 6, ""america"":..."
2,2,"{""calpine"": 2, ""daily"": 2, ""doc"": 1, ""gas"": 2,..."
3,3,"{""accounting"": 1, ""already"": 2, ""am"": 4, ""arra..."
4,4,"{""allen"": 3, ""allocated"": 2, ""allocation"": 3, ..."
5,5,"{""active"": 1, ""advice"": 1, ""also"": 1, ""and"": 3..."
...,...,...
33711,33711,"{""aiready"": 1, ""aveyoumone"": 1, ""can"": 1, ""cas..."
33712,33712,"{""about"": 1, ""affordable"": 1, ""after"": 1, ""all..."
33713,33713,"{""also"": 1, ""are"": 2, ""be"": 1, ""because"": 1, ""..."
33714,33714,"{""bed"": 1, ""bloow"": 1, ""here"": 1, ""how"": 1, ""i..."


In [33]:
test_out

,Message ID,features_json
0,0,"{""christmas"": 1, ""farm"": 1, ""pictures"": 1, ""tr..."
30,30,"{""all"": 1, ""am"": 1, ""and"": 5, ""any"": 3, ""at"": ..."
60,60,"{""any"": 1, ""as"": 1, ""backup"": 1, ""be"": 1, ""by""..."
90,90,"{""actuals"": 1, ""enron"": 3, ""for"": 1, ""hpl"": 2,..."
120,120,"{""above"": 1, ""according"": 1, ""actuals"": 1, ""al..."
...,...,...
33570,33570,"{""act"": 1, ""appears"": 1, ""as"": 1, ""be"": 1, ""be..."
33600,33600,"{""an"": 6, ""ani"": 3, ""anic"": 1, ""ar"": 6, ""are"":..."
33630,33630,"{""about"": 1, ""all"": 1, ""and"": 5, ""appreciate"":..."
33660,33660,"{""all"": 1, ""and"": 6, ""approval"": 1, ""approved""..."


In [34]:
train_out.to_csv("train-features.tsv", sep="\t", index=False)
test_out.to_csv("test-features.tsv", sep="\t", index=False)

Submit "train-features.tsv" and "test-features.tsv" in Gradescope.

Hint: these features will be graded based on the test accuracy of a logistic regression based on the training features.
This is to make sure that your feature set is not degenerate; you do not need to compute this regression yourself.
You can separately assess your feature quality based on your results in part 6.

## Part 3: Compute Conditional Probabilities

Based on your training data, compute appropriate conditional probabilities for use with naïve Bayes.
Use of additive smoothing with $\alpha=1$ to avoid zeros.


In [22]:
# YOUR CHANGES HERE
train_features = pd.read_csv("train-features.tsv", sep="\t")[["Message ID", "features_json"]]
labels_df = pd.read_csv("enron_spam_data.zip")[["Message ID", "Spam/Ham"]]

In [23]:
new_train_df = train_features.merge(labels_df, on="Message ID", how="left")

In [24]:
ham_counts = Counter()
spam_counts = Counter()

In [26]:
new_train_df

,Message ID,features_json,Spam/Ham
0,1,"{""about"": 1, ""advises"": 1, ""am"": 6, ""america"":...",ham
1,2,"{""calpine"": 2, ""daily"": 2, ""doc"": 1, ""gas"": 2,...",ham
2,3,"{""accounting"": 1, ""already"": 2, ""am"": 4, ""arra...",ham
3,4,"{""allen"": 3, ""allocated"": 2, ""allocation"": 3, ...",ham
4,5,"{""active"": 1, ""advice"": 1, ""also"": 1, ""and"": 3...",ham
...,...,...,...
32587,33711,"{""aiready"": 1, ""aveyoumone"": 1, ""can"": 1, ""cas...",spam
32588,33712,"{""about"": 1, ""affordable"": 1, ""after"": 1, ""all...",spam
32589,33713,"{""also"": 1, ""are"": 2, ""be"": 1, ""because"": 1, ""...",spam
32590,33714,"{""bed"": 1, ""bloow"": 1, ""here"": 1, ""how"": 1, ""i...",spam


In [21]:
train_df.columns

Index(['Message ID', 'Subject', 'Message', 'Spam/Ham_x', 'Date', 'text',
       'features_json', 'Spam/Ham_y'],
      dtype='object')

In [ ]:
# aggregate counts
for _, row in new_train_df.iterrows():
    features = json.loads(row["features_json"])
    if str(row["Spam/Ham"]).strip().lower() == "spam":
        spam_counts.update(features)
    else:
        ham_counts.update(features)

In [35]:
# compute conditional probabilities with additive smoothing

vocab_size = len(set(ham_counts) | set(spam_counts))
alpha = 1.0
num_ham = sum(ham_counts.values())
num_spam = sum(spam_counts.values())

In [37]:
output = []
for f in sorted(set(ham_counts) | set(spam_counts)):
    ham_p  = (ham_counts[f]  + alpha) / (num_ham  + alpha * vocab_size)
    spam_p = (spam_counts[f] + alpha) / (num_spam + alpha * vocab_size)
    output.append({"feature": f, "ham_probability": ham_p, "spam_probability": spam_p})


In [39]:
output_df = pd.DataFrame(output)

In [40]:
output_df

,feature,ham_probability,spam_probability
0,aa,5.394445e-05,5.654529e-05
1,aaa,4.187469e-06,5.838915e-06
2,aaaa,2.463217e-07,9.219340e-07
3,aaaacy,2.463217e-07,6.146227e-07
4,aaaahhhhhh,4.926434e-07,3.073113e-07
...,...,...,...
141203,zzyudgpd,2.463217e-07,6.146227e-07
141204,zzzglvaa,2.463217e-07,6.146227e-07
141205,zzzxlqbha,2.463217e-07,6.146227e-07
141206,zzzz,2.463217e-07,9.526651e-06


Save the conditional probabilities in a file "feature-probabilities.tsv" with columns feature, ham_probability and spam_probability.

In [41]:
# YOUR CHANGES HERE

output_df.to_csv("feature-probabilities.tsv", sep="\t", index=False)

Submit "feature-probabilities.tsv" in Gradescope.

## Part 4: Implement a Naïve Bayes Classifier

Implement a naïve Bayes classifier based on your previous feature probabilities.

In [46]:
# YOUR CHANGES HERE

labels = pd.read_csv("enron_spam_data.zip", usecols=["Message ID","Spam/Ham"])
labels = labels[labels["Message ID"] % 30 != 0]
train_final = train_features.merge(labels, on="Message ID", how="inner")
probs = pd.read_csv("feature-probabilities.tsv", sep="\t")

In [45]:
y = train_final["Spam/Ham"].str.strip().str.lower()
p_ham_prior  = (y == "ham").mean()
p_spam_prior = (y == "spam").mean()

In [47]:
ham_p  = dict(zip(probs.feature, probs.ham_probability))
spam_p = dict(zip(probs.feature, probs.spam_probability))
vocab_size = len(set(ham_p) | set(spam_p))

In [50]:
alpha = 1.0
ham_counts = Counter()
spam_counts = Counter()

In [52]:
for _, r in train_final.iterrows():
    d = json.loads(r["features_json"])
    (spam_counts if r["Spam/Ham"].strip().lower()=="spam" else ham_counts).update(d)

num_ham = sum(ham_counts.values())
num_spam = sum(spam_counts.values())

p_ham_smooth  = alpha / (num_ham  + alpha * vocab_size)
p_spam_smooth = alpha / (num_spam + alpha * vocab_size)

In [59]:
def score_json(feats_json):
    d = json.loads(feats_json)
    log_h = np.log(p_ham_prior);  log_s = np.log(p_spam_prior)
    for tok, c in d.items():
        log_h += c * np.log(ham_p.get(tok,  p_ham_smooth))
        log_s += c * np.log(spam_p.get(tok, p_spam_smooth))
    m = max(log_h, log_s)
    ph = np.exp(log_h - m); ps = np.exp(log_s - m)
    z = ph + ps
    return ph/z, ps/z

Save your prediction probabilities to "train-predictions.tsv" with columns Message ID, ham and spam.

In [60]:
# YOUR CHANGES HERE

pred = train_final[["Message ID","features_json"]].copy()
pred[["ham","spam"]] = pred["features_json"].apply(lambda s: pd.Series(score_json(s)))

In [61]:
pred = pred.drop(columns=["features_json"]).sort_values("Message ID")


In [62]:
pred

,Message ID,ham,spam
0,1,1.000000e+00,3.224627e-166
1,2,1.000000e+00,1.415113e-12
2,3,1.000000e+00,4.771810e-149
3,4,1.000000e+00,9.275319e-143
4,5,1.000000e+00,6.106032e-38
...,...,...,...
32587,33711,2.241310e-18,1.000000e+00
32588,33712,3.317017e-34,1.000000e+00
32589,33713,4.881864e-16,1.000000e+00
32590,33714,4.137717e-03,9.958623e-01


In [63]:
pred.to_csv("train-predictions.tsv", sep="\t", index=False)

Submit "train-predictions.tsv" in Gradescope.

## Part 5: Predict Spam Probability for Test Data

Use your previous classifier to predict spam probability for the test data.

In [ ]:
# YOUR CHANGES HERE
# set up as we did in part 4 to utilize classifier

test_feats = pd.read_csv("test-features.tsv", sep="\t")
labels = pd.read_csv("enron_spam_data.zip", usecols=["Message ID","Spam/Ham"])

In [67]:
labels_test = labels[labels["Message ID"] % 30 == 0]
test_df = test_feats.merge(labels_test, on="Message ID", how="left")

In [69]:
ham_p  = dict(zip(probs.feature, probs.ham_probability))
spam_p = dict(zip(probs.feature, probs.spam_probability))
vocab_size = len(set(ham_p) | set(spam_p))

In [70]:
train_text = labels[labels["Message ID"] % 30 != 0]
p_ham_prior  = (train_text["Spam/Ham"].str.lower() == "ham").mean()
p_spam_prior = (train_text["Spam/Ham"].str.lower() == "spam").mean()

In [71]:
train_df = train_features.merge(train_text, on="Message ID", how="inner")

In [73]:
ham_counts = Counter()
spam_counts = Counter()

In [74]:
for _, r in train_df.iterrows():
    d = json.loads(r["features_json"])
    (spam_counts if r["Spam/Ham"].strip().lower()=="spam" else ham_counts).update(d)

In [75]:
num_ham = sum(ham_counts.values())
num_spam = sum(spam_counts.values())

alpha = 1.0

p_ham_smooth  = alpha / (num_ham  + alpha * vocab_size)
p_spam_smooth = alpha / (num_spam + alpha * vocab_size)

In [76]:
# perform prediction

def predict_probs(features_json):
    feats = json.loads(features_json)
    log_h = np.log(p_ham_prior);  log_s = np.log(p_spam_prior)
    for tok, c in feats.items():
        log_h += c * np.log(ham_p.get(tok,  p_ham_smooth))
        log_s += c * np.log(spam_p.get(tok, p_spam_smooth))

    m = max(log_h, log_s)
    prop_ham = np.exp(log_h - m)
    prop_spam = np.exp(log_s - m)
    normalization = prop_ham + prop_spam
    return prop_ham/normalization, prop_spam/normalization

In [77]:
test_pred = test_df[["Message ID","features_json"]].copy()
test_pred[["ham","spam"]] = test_pred["features_json"].apply(lambda s: pd.Series(predict_probs(s)))

In [78]:
test_pred = test_pred.drop(columns=["features_json"]).sort_values("Message ID")

In [79]:
test_pred

,Message ID,ham,spam
0,0,5.815942e-02,9.418406e-01
1,30,1.000000e+00,2.111283e-79
2,60,1.000000e+00,1.709230e-12
3,90,1.000000e+00,7.266323e-33
4,120,1.000000e+00,2.456632e-169
...,...,...,...
1119,33570,1.454380e-12,1.000000e+00
1120,33600,0.000000e+00,1.000000e+00
1121,33630,8.866389e-48,1.000000e+00
1122,33660,1.657485e-70,1.000000e+00


Save your prediction probabilities in "test-predictions.tsv" with the same columns as "train-predictions.tsv".

In [80]:
# YOUR CHANGES HERE

test_pred.to_csv("test-predictions.tsv", sep="\t", index=False)


Submit "test-predictions.tsv" in Gradescope.

## Part 6: Construct ROC Curve

For every probability threshold from 0.01 to .99 in increments of 0.01, compute the false and true positive rates from the test data using the spam class for positives.
That is, if the predicted spam probability is greater than or equal to the threshold, predict spam.

In [82]:
# YOUR CHANGES HERE

emails_test = labels[labels["Message ID"] % 30 == 0]
final_df = test_pred.merge(emails_test, on="Message ID", how='left')

In [83]:
final_df["is_spam"] = (df["Spam/Ham"].str.strip().str.lower() == "spam").astype(int)

In [84]:
thresholds = np.arange(0.01, 1.00, 0.01)
roc_data = []

In [85]:
for t in thresholds:
    final_df["predicted"] = (final_df["spam"] >= t).astype(int)

    TP = ((final_df["is_spam"] == 1) & (final_df["predicted"] == 1)).sum()
    FP = ((final_df["is_spam"] == 0) & (final_df["predicted"] == 1)).sum()
    TN = ((final_df["is_spam"] == 0) & (final_df["predicted"] == 0)).sum()
    FN = ((final_df["is_spam"] == 1) & (final_df["predicted"] == 0)).sum()
    
    # true positive rate
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0 
    # false positive rate
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # false positive rate

    roc_data.append({"threshold": round(t, 2), 
                     "false_positive_rate": FPR, 
                     "true_positive_rate": TPR})

In [86]:
roc_df = pd.DataFrame(roc_data)

In [87]:
roc_df

,threshold,false_positive_rate,true_positive_rate
0,0.01,0.523132,0
1,0.02,0.523132,0
2,0.03,0.522242,0
3,0.04,0.522242,0
4,0.05,0.521352,0
...,...,...,...
94,0.95,0.503559,0
95,0.96,0.501779,0
96,0.97,0.501779,0
97,0.98,0.500890,0


Save this data in a file "roc.tsv" with columns threshold, false_positive_rate and true_positive rate.

In [89]:
# YOUR CHANGES HERE

roc_df.to_csv("roc.tsv", sep="\t", index=False)

Submit "roc.tsv" in Gradescope.

## Part 7: Signup for Gemini API Key

Create a free Gemini API key at https://aistudio.google.com/app/api-keys.
You will need to do this with a personal Google account - it will not work with your BU Google account.
This will not incur any charges unless you configure billing information for the key.

You will be asked to start a Gemini free trial for week 11.
This will not incur any charges unless you exceed expected usage by an order of magnitude.


No submission needed.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.